In [9]:
import torch
from torch import nn
from d2l import torch as d2l

In [10]:
#读取数据
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

In [11]:
#转变为符合格式的图片
class Reshape(torch.nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)

#Flatten就是把输入批量的那一维度不变，其它维度拉成一个向量
net=nn.Sequential(
    Reshape(),
    nn.Conv2d(1,6,kernel_size=5,padding=2),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Conv2d(6,16,kernel_size=5),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5,120),
    nn.Sigmoid(),
    nn.Linear(120,84),
    nn.Sigmoid(),
    nn.Linear(84,10)
    )

In [12]:
#查看层中各输入输出维度的大小
X=torch.rand(size=(1,1,28,28),dtype=torch.float32)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)
X

Reshape output shape:	 torch.Size([1, 1, 28, 28])
Conv2d output shape:	 torch.Size([1, 6, 28, 28])
Tanh output shape:	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape:	 torch.Size([1, 6, 14, 14])
Conv2d output shape:	 torch.Size([1, 16, 10, 10])
Tanh output shape:	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape:	 torch.Size([1, 16, 5, 5])
Flatten output shape:	 torch.Size([1, 400])
Linear output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


tensor([[ 0.2728, -0.3562,  0.0868, -0.4422,  0.1509,  0.3348,  0.7059,  0.0368,
          0.3686, -0.1758]], grad_fn=<AddmmBackward>)

In [13]:
def evaluate_accuracy_gpu(net,data_iter,device=None):
    if isinstance(net,torch.nn.Module):
        net.eval()#设置为评估模式
        if not device:
            device=next(iter(net.parameters())).device
    #正确预测的数量和总预测的数量
    metric = d2l.Accumulator(2)
    for X,y in data_iter:
        if isinstance(X,list):
            #BERT微调所需要的
            X = [x.to(device) for x in X]
        else:
            X=X.to(device)
        y=y.to(device)
        metric.add(d2l.accuracy(net(X),y),y.numel())
    return metric[0]/metric[1]

In [14]:
#针对不同的设计网络的时候优化器和loss函数是自己定义的
def Train(net,train_iter,test_iter,num_epochs,lr,device):
    def init_weight(m):
        if(type(m)==nn.Linear or type(m)==nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)#均匀分布初始化权重

    net.apply(init_weight)
    #打印正在运行的是GPU还是CPU
    print('training on',device)
    net.to(device)#网络已经放到GPU上了
    #定义优化器
    optimizer=torch.optim.SGD(net.parameters(),lr=lr)
    #定义损失函数：分类问题使用交叉熵
    loss=nn.CrossEntropyLoss()

    #画出训练图像
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)

    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，范例数
        metric = d2l.Accumulator(3)
        net.train()#模型训练前的固定语句
        for i,(X,y) in enumerate(train_iter):
            timer.start()#计时器
            optimizer.zero_grad()
            #数据放到GPU上
            X,y=X.to(device),y.to(device)
            y_hat=net(X)#前向传播
            l=loss(y_hat,y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

In [15]:
lr,num_epochs=0.9,10
Train(net,train_iter,test_iter,num_epochs,lr,torch.device('cuda'))
torch.save(net,'./LeNet.pth')

loss 0.296, train acc 0.889, test acc 0.873
17213.6 examples/sec on cuda


In [16]:
#实验的一个发现:将Sigmoid改为Tanh后精度提升接近6各百分点
